In [2]:
#QUERY_URL = "//div[@class='separator']//a/@href"
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"


In [3]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import json
import validators
from time import sleep
from functools import partial


In [10]:
from os import mkdir, listdir
from os.path import join, isfile, isdir, exists, abspath
from tqdm import tqdm, trange
import re
from functools import reduce
from PIL import Image


In [5]:
class Crawler:

    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    def __init__(self):

        # ruta XPATH hacia los enlaces que queremos recopilar
        self.query_extract_all_href = QUERY_URL

        # variable que permite listar todos los enlaces recopilado
        self.all_json_to_send = []
        pass

    def new_session_get(self):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        return partial(session.get, headers=header)

    def start_request(self, url):
        session_get = self.new_session_get()
        response = session_get(url)
        return response

    def parse(self, response):
        self.all_json_to_send = []
        all_extracted_href = response.html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send = dict()
            #path = "https://www.hindustantimes.com"
            # json_to_send["url"]=path+href
            json_to_send["url"] = href
            self.all_json_to_send.append(json_to_send)


In [108]:
SEED_CRAWL = 'https://www.cnnchile.com/category/cultura/page/'
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
MAX_PAGE_CULTURA = 32
LAST_ID = 17798719
STARTING_ID_CULTURA = 18000000

crawler = Crawler()

In [9]:
url_noticias = []
for page in trange(MAX_PAGE_CULTURA, desc='url noticias'):
    response = crawler.start_request(SEED_CRAWL + str(page))
    crawler.parse(response)
    noticias = [obj['url']
                for obj in crawler.all_json_to_send if 'cultura' in obj['url']]
    url_noticias += noticias

len(url_noticias)


364

In [97]:
noticia_a = url_noticias[4]
response = crawler.start_request(noticia_a)
noticia_dict = {'url': noticia_a}
noticia_a


'https://www.cnnchile.com/cultura/muere-almudena-grandes-a-los-61-anos_20211127/'

In [98]:
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
noticia_dict['title'] = response.html.xpath(QUERY_TITLE)[0]
response.html.xpath(QUERY_TITLE)[0]


'Muere la reconocida escritora española Almudena Grandes a los 61 años'

In [99]:
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"
noticia_dict['text'] = re.sub(
    u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
noticia_dict['text']

'La escritora murió a los 61 años a causa del cáncer que dio a conocer en una  columna publicada en el diario El País. El presidente del gobierno español, Pedro Sánchez y el artista Alejandro Sanz lamentaron su partida a través de redes sociales. '

In [100]:
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"
new_lines = response.html.xpath(QUERY_CONTENT)[0].text.splitlines()
re.match(r'^(.*)(– )(.*)', new_lines[0]).groups()

('(CNN Español) ',
 '– ',
 'La reconocida escritora española Almudena Grandes falleció este sábado, así lo confirmó El País, diario en el cual Grandes se desempeñaba como columnista.')

In [103]:
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"
lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
new_lines = [re.sub(u'\xa0', u' ', line)
             for line in lines if not line.startswith('Lee también:')]
new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
noticia_dict['text'] += ' '.join(new_lines)

In [104]:
noticia_dict

{'url': 'https://www.cnnchile.com/cultura/muere-almudena-grandes-a-los-61-anos_20211127/',
 'title': 'Muere la reconocida escritora española Almudena Grandes a los 61 años',
 'text': 'La escritora murió a los 61 años a causa del cáncer que dio a conocer en una  columna publicada en el diario El País. El presidente del gobierno español, Pedro Sánchez y el artista Alejandro Sanz lamentaron su partida a través de redes sociales. La reconocida escritora española Almudena Grandes falleció este sábado, así lo confirmó El País, diario en el cual Grandes se desempeñaba como columnista. Grandes, de 61 años, murió debido a un cáncer que ella mismo dio a conocer a través de una de sus columnas. La autora de “Las edades de Lulú” y “La madre de Frankestein” era una de las escritoras españolas más prolíferas de los últimos años. Fue ganadora de varios galardones, entre ellos el Premio Nacional de Narrativa en 2018 en España y el Premio Iberoamericano de Novela Elena Poniatowska. Su muerte causó la r

In [110]:
from time import sleep
noticias_cultura = []
_id = STARTING_ID_CULTURA
for i, url in enumerate(tqdm(url_noticias)):
    _id = STARTING_ID_CULTURA + i
    response = crawler.start_request(url)
    _dict = {'id': _id,
             'url':  url}
    _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
    _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
    lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
    new_lines = [re.sub(u'\xa0', u' ', line)
                 for line in lines if not line.startswith('Lee también:')]
    new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
    _dict['text'] += ' '.join(new_lines)
    noticias_cultura.append(_dict)
    sleep(5)
noticias_cultura[:3]


100%|██████████| 364/364 [36:14<00:00,  5.97s/it]


[{'id': 18000000,
  'url': 'https://www.cnnchile.com/cultura/musico-joan-manuel-serrat-anuncio-retiro_20211202/',
  'title': 'Joan Manuel Serrat anunció su retiro',
  'text': 'La despedida del artista comenzará en el Beacon Theater de Nueva York el 27 de abril de 2022 y finalizará en el Palau Sant Jordi de Barcelona el 23 de diciembre. Además, el 8 de diciembre podrían conocerse más detalles de esta gira que incluirá también a Sudamérica.El cantautor español Joan Manuel Serrat, de 77 años, anunció su retiro de los escenarios. La noticia fue conocida a través de una entrevista con el diario El País, donde compositor expresó que la crisis sanitaria jugó un rol determinante en su decisión, ya que “no me gustó sentirme despedido por una plaga”. “El encierro al que nos llevó esta pandemia que aún dura provocó la imposibilidad de continuar el oficio de cantar en público. Y, también, por la necesidad de recuperar la vida familiar, cumplir con cuestiones íntimas y necesarias”, confesó. Y para 

In [113]:
for i, _dict in enumerate(noticias_cultura):
    _dict['id'] = STARTING_ID_CULTURA + i
#stop_idx = next((i for i, line in enumerate(new_lines) if line.startswith('Tags:')), None)
# new_lines[:stop_idx]
noticias_cultura[5]


{'id': 18000005,
 'url': 'https://www.cnnchile.com/cultura/jaar-rosenfeld-matta-clark-bienal-artes-mediales-santiago_20211126/',
 'title': 'Jaar, Rosenfeld y Matta-Clark encabezan la muestra de la 15º Bienal de Artes Mediales de Santiago',
 'text': 'Visualidad, multimedialidad y tecnología, son algunas de las bases de muestra que desde 1993 expone a creadores locales e internacionales. En esta edición, que comienza este viernes, destaca la instalación de Alfredo Jaar, una experiencia sinfónica inmersiva que utiliza los sonidos de cientos de recién nacidos en pandemia en Chile, algo que, destacó, coincide con el nacimiento de la Convención Constitucional.Este viernes se dio inicio oficial a la 15º versión de la Bienal de Artes Mediales de Santiago, con la apertura de una muestra en el Museo Nacional de Bellas de Artes (MNBA), encabezada por obras de los artistas visuales Alfredo Jaar, Lotty Rosenfeld, Gordon Matta-Clark y el director de cine Raúl Ruiz, además de una retrospectiva al tra

In [115]:
import pandas as pd
df = pd.DataFrame(noticias_cultura)
df


,id,url,title,text
0,18000000,https://www.cnnchile.com/cultura/musico-joan-m...,Joan Manuel Serrat anunció su retiro,La despedida del artista comenzará en el Beaco...
1,18000001,https://www.cnnchile.com/cultura/rihanna-obtie...,Rihanna obtiene la designación de “heroína nac...,"La cantante fue homenajeada en su país natal, ..."
2,18000002,https://www.cnnchile.com/cultura/400-peliculas...,Desde “El Húsar de la Muerte” hasta la actuali...,La cineteca de la casa de estudios celebra 60 ...
3,18000003,https://www.cnnchile.com/cultura/cordillera-su...,"“La cordillera de los sueños”, de Patricio Guz...",La 36ª edición de los Premios Goya se realizar...
4,18000004,https://www.cnnchile.com/cultura/muere-almuden...,Muere la reconocida escritora española Almuden...,La escritora murió a los 61 años a causa del c...
...,...,...,...,...
359,18000359,https://www.cnnchile.com/cultura/calamaro-mon-...,“Dios los Cría”: Calamaro invita a Mon Laferte...,"La chilena interpretará ""Tantas veces"" junto a..."
360,18000360,https://www.cnnchile.com/cultura/paloma-mami-v...,“Qué Weá”: Paloma Mami se reivindica como chil...,La cantante participó en la dirección del vide...
361,18000361,https://www.cnnchile.com/cultura/ellen-degener...,Ellen DeGeneres informó que planea terminar su...,"La popular animadora explicó que ""cuando eres ..."
362,18000362,https://www.cnnchile.com/cultura/entrevista-do...,Los poemas en la UCI del Dr. Glenn Hernández: ...,Casi un año antes del primer muerto por la pan...


In [116]:
df.to_csv('extra_cultura.csv', index=False)


In [118]:
# trabajo manual, buscar tags que puedan estar relacionados con ecologia y planeta
tags_ecologia_planeta = [('animales', 8),            # total 19
                         ('maltrato-animal', 4),     # total 6
                         ('especies', 2),            # total 2
                         ('crisis-climatica', 4),    # total 4
                         ('cambio-climatico', 4),    # total 34
                         ('emergencia-climatica', 3),# total 3
                         ('vida-silvestre', 1),      # total 1
                         ('humedales', 1),            # total 1
                         ('fauna', 5),               # total 5
                         ('medioambiente', 5),       # total 8
                         ('medio-ambiente', 5),      # total 22
                         #('greta_thornberryXD', 7),
                         ]

In [120]:
# obtener las url de las noticias relacionadas con la categoria
url_noticias = []
STARTING_ID_PLANETA = 19000000

for (tag_name, num_pages) in tqdm(tags_ecologia_planeta):
    SEED_CRAWL = f'https://www.cnnchile.com/tag/{tag_name}/page/'
    QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
    crawler = Crawler()
    for page in range(1, num_pages+1):
        response = crawler.start_request(SEED_CRAWL + str(page))
        crawler.parse(response)
        noticias = [obj['url'] for obj in crawler.all_json_to_send]
        url_noticias += noticias

print(len(url_noticias))
url_noticias = list(set(url_noticias))  # eliminar posibles duplicados
len(url_noticias)


100%|██████████| 11/11 [00:39<00:00,  3.60s/it]

420


383

In [124]:
# obtener las noticias en si
noticias_planeta = []
for i, url in enumerate(tqdm(url_noticias)):
    _id = STARTING_ID_PLANETA + i
    response = crawler.start_request(url)
    _dict = {'id': _id,
             'url':  url}
    _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
    _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
    lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
    new_lines = [re.sub(u'\xa0', u' ', line)
                 for line in lines if not line.startswith('Lee también:')]
    new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
    _dict['text'] += ' '.join(new_lines)
    noticias_planeta.append(_dict)
    sleep(5) # porsiaca
noticias_planeta

  0%|          | 0/383 [00:05<?, ?it/s]


[{'id': 19000000,
  'url': 'https://www.cnnchile.com/mundo/greta-thunberg-foro-economico-mundial-2020-davos-cambio-climatico_20200121/',
  'title': 'Greta Thunberg en Davos: Me han escuchado, pero no han hecho nada respecto al cambio climático',
  'text': '"Nuestra casa está aún en llamas y su inacción aviva las llamas cada hora", dijo la joven activista sueca en el Foto Económico Mundial, parafraseando su propio discurso ante el mismo auditorio el año pasado, una frase que le dio la vuelta al mundo y que, un año después, sigue vigente.Este martes partió el Foro Económico Mundial en Davos, Suiza. La edición 2020 es también el 50 aniversario de la cuestionada, polémica e influyente agenda del poder económico global. Este año con una potente agenda climática, al igual que en 2019, recibió a Greta Thunberg entre sus expositores. La joven activista contra el cambio climático no decepcionó a sus seguidores. Mantuvo un discurso reflexivo y directo sobre la causa por la que ha cruzado el plan